In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crime-data/crime data.csv


In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [13]:
data = pd.read_csv('/kaggle/input/crime-data/crime data.csv')

In [14]:
data.drop(columns=['Process 9','Process 10','Process 11'])

,Case ID,Type of cyber Attack,Process 1,Process 2,Process 3,Process 4,Process 5,Process 6,Process 7,Process 8
0,1,Malware,Reconnaissance:,Delivery,Exploitation,Installation,Command and Control (C2),Propagation,Evasion,Execution of Payload
1,2,Phishing,Target Selection,Message Crafting,Social Engineering,Delivery,Clicking the Link,Payload Delivery,Data Collection,Post-Attack Activities
2,3,Ransomware,Delivery,Execution,Encryption,Ransom Note,Payment,Decryption (optional),NaN,NaN
3,4,SQL injection,Identifying Vulnerable Points,Understanding the Database Structure,Crafting Malicious Input,Injection,Exploitation,Data Extraction or Modification,NaN,NaN
4,5,Denial-of-service (DoS),Target Selection,Resource Exhaustion,Service Disruption,No Further Action,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
104,106,BadUSB Attacks,Malicious Firmware Installation,Device Connection,Automated Exploitation,NaN,NaN,NaN,NaN,NaN
105,107,Social Media Impersonation,Target Selection,Profile Creation,Engagement,NaN,NaN,NaN,NaN,NaN
106,108,Bait-and-Switch Attacks,Initial Legitimate Interaction,Substitution,Exploitation,NaN,NaN,NaN,NaN,NaN
107,109,Firmware Rootkits,Identification of Target Devices,Exploitation,Persistence,NaN,NaN,NaN,NaN,NaN


In [15]:
data[data['Process 1']=='Monitoring Physical Side Channels']

,Case ID,Type of cyber Attack,Process 1,Process 2,Process 3,Process 4,Process 5,Process 6,Process 7,Process 8,Process 9,Process 10,Process 11
57,59,Side-Channel Attacks,Monitoring Physical Side Channels,Monitoring Physical Side Channels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
crime_types = data.iloc[:,1:2]
crime_types = crime_types.values.tolist()
crime_process_steps = data.iloc[:, 2:].values.tolist()
lowercase_steps = [[step.lower() if isinstance(step, str) else str(step) for step in sublist] for sublist in crime_process_steps]
flattened_steps = [step for sublist in lowercase_steps for step in sublist]

In [17]:
vectorizer = TfidfVectorizer()
crime_process_vectors = vectorizer.fit_transform(flattened_steps)

In [18]:
knn = NearestNeighbors(n_neighbors=5)
knn.fit(crime_process_vectors)


NearestNeighbors()

# errored one

In [19]:
def predict_next_steps(crime_type, crime_process_steps):
    # Assuming you have previously defined vectorizer, knn, and data
    
    crime_process_steps_vector = vectorizer.transform(crime_process_steps)
    
    neighbors = knn.kneighbors(crime_process_steps_vector, return_distance=False)

    print("Neighbors:", neighbors)

    if not any(neighbors[0]):
        print("No neighbors found.")
        return []

    max_index = max(neighbors[0])
    if max_index >= len(data):
        print(f"Index out of bounds. Max index: {max_index}, Data length: {len(data)}")
        return []

    neighboring_crime_processes = data.iloc[neighbors[0], 2:].values.tolist()

    next_steps = []
    for crime_process in neighboring_crime_processes:
        for step in crime_process:
              if step not in crime_process_steps:
                next_steps.append(step)


    return next_steps

# Example usage
crime_type = 'Phishing'
crime_process_steps = ['Target Selection', 'Message Crafting']
next_steps = predict_next_steps(crime_type, crime_process_steps)
print(next_steps)

Neighbors: [[ 57  11  44 275  79]
 [ 12  35 650   1   0]]
Index out of bounds. Max index: 275, Data length: 109
[]


# all the steps are presented

In [21]:
def predict_next_steps(crime_type, crime_process_steps):
    # Assuming you have previously defined vectorizer, knn, and data
    
    crime_process_steps_vector = vectorizer.transform(crime_process_steps)
    
    neighbors = knn.kneighbors(crime_process_steps_vector, return_distance=False)

    if not any(neighbors[0]):
        print("No neighbors found.")
        return []

    # Use modulo to wrap around indices
    neighboring_indices = [index % len(data) for index in neighbors[0]]

    neighboring_crime_processes = data.iloc[neighboring_indices, 2:].values.tolist()

    next_steps = []
    for crime_process in neighboring_crime_processes:
        for step in crime_process:
            if not pd.isna(step) and step not in crime_process_steps:
                next_steps.append(step)

    return next_steps

# Example usage
crime_type = 'Malware'
crime_process_steps = ['Exploitation','Delivery', 'Installation']
next_steps = predict_next_steps(crime_type, crime_process_steps)
print(next_steps)


['Malicious Firmware Installation', 'Device Connection', 'Automated Exploitation', 'Introduction of Malicious Code', 'Interception of Function Calls', 'Persistence', 'Execution', 'Encryption', 'Ransom Note', 'Payment', 'Decryption (optional)', 'Network Interception', 'Packet Sniffing', 'Session Hijacking', 'Data Interception', 'SSL Stripping', 'Spoofing', 'FTP Connection Setup', 'Bounce Target Identification', 'Bounce Attack Execution']


# one step is predicted

In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

data = pd.read_csv('/kaggle/input/crime-data/crime data.csv')
data.drop(columns=['Process 9', 'Process 10', 'Process 11'])

crime_types = data.iloc[:, 1:2]
crime_types = crime_types.values.tolist()

crime_process_steps = data.iloc[:, 2:].values.tolist()

lowercase_steps = [[step.lower() if isinstance(step, str) else str(step) for step in sublist] for sublist in crime_process_steps]

flattened_steps = [step for sublist in lowercase_steps for step in sublist]

vectorizer = TfidfVectorizer()
crime_process_vectors = vectorizer.fit_transform(flattened_steps)

knn = NearestNeighbors(n_neighbors=6)
knn.fit(crime_process_vectors)


def predict_next_step(crime_type, crime_process_steps):
    # Assuming you have previously defined vectorizer, knn, and data

    crime_process_steps_vector = vectorizer.transform(crime_process_steps)

    neighbors = knn.kneighbors(crime_process_steps_vector, return_distance=False)[0]

    if not any(neighbors):
        print("No neighbors found.")
        return None

    # Use modulo to wrap around indices
    neighboring_index = neighbors[0] % len(data)

    neighboring_crime_process = data.iloc[neighboring_index, 2:].values

    next_step = None
    for step in neighboring_crime_process:
        if not pd.isna(step) and step not in crime_process_steps:
            next_step = step
            break

    return next_step


# Example usage
crime_type = 'Malware'
crime_process_steps = ['Exploitation', 'Delivery','Installation']
next_step = predict_next_step(crime_type, crime_process_steps)
print(next_step)


Execution


#  USING LSTM

In [ ]:
# Create a vocabulary from the flattened steps
vocabulary = list(set(flattened_steps))

# Create a TfidfVectorizer to convert steps into numerical vectors
vectorizer = TfidfVectorizer(vocabulary=vocabulary)

# Convert crime process steps into numerical vectors
crime_process_vectors = vectorizer.fit_transform(flattened_steps)

# Create a dictionary to map steps to their corresponding vector indices
step_indices = {step: index for index, step in enumerate(vocabulary)}

# Convert crime process steps into sequences of indices
step_sequences = [[step_indices[step] for step in sublist] for sublist in lowercase_steps]

# Pad sequences to the same length
max_length = max(len(sequence) for sequence in step_sequences)
step_sequences_padded = pad_sequences(step_sequences, maxlen=max_length, padding='post')

# Create an LSTM model
model = Sequential()
model.add(Embedding(len(vocabulary), 128, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(len(vocabulary), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(step_sequences_padded, crime_types, epochs=10, batch_size=64)

def predict_next_step(crime_type, crime_process_steps):
    # Convert crime process steps into numerical vectors
    crime_process_vectors = vectorizer.transform(crime_process_steps)

    # Convert crime process vectors into sequences of indices
    step_sequences = [[step_indices[step] for step in sublist] for sublist in lowercase_steps]

    # Pad sequences to the same length
    step_sequences_padded = pad_sequences(step_sequences, maxlen=max_length, padding='post')

    # Predict the next step
    next_step_predictions = model.predict(step_sequences_padded)

    # Convert next step predictions into step indices
    next_step_indices = np.argmax(next_step_predictions, axis=1)

    # Convert next step indices into steps
    next_steps = [vocabulary[index] for index in next_step_indices]

    return next_steps

# Example usage
crime_type = 'Malware'
crime_process_steps = ['Exploitation', 'Delivery', 'Installation']
next_steps = predict_next_step(crime_type, crime_process_steps)
print(next_steps)


In [ ]:

# Preprocessing
tokenizer = Tokenizer(filters='')

for text in data.values.flatten():
    if isinstance(text, str):
        tokenizer.fit_on_texts([text])
    else:
        # Ignore non-string elements
        pass

total_words = len(tokenizer.word_index) + 1

input_sequences = []
max_seq_length = 0  # Initialize max sequence length

for row in data.iterrows():
    row_values = row[1].values[1:]
    curr_seq_length = len(row_values)
    max_seq_length = max(max_seq_length, curr_seq_length)

    for i in range(curr_seq_length - 1):
        sequence = row_values[:i + 1]
        next_step = row_values[i + 1]

        # Pad the sequence with zeros to match the maximum length
        padded_sequence = sequence + ['0'] * (max_seq_length - len(sequence))
        input_sequences.append((padded_sequence, next_step))

input_sequences = [(tokenizer.texts_to_sequences(seq), tokenizer.texts_to_sequences([next_step]))
                    for seq, next_step in input_sequences]

X = np.array([item[0][0] for item in input_sequences])
y = np.array([item[1][0][0] for item in input_sequences])



'''
# Create input sequences and labels
input_sequences = []
for row in data.iterrows():
    row_values = row[1].values[1:]
    for i in range(len(row_values)):
        if i < len(row_values) - 1:
            sequence = row_values[:i + 1]
            next_step = row_values[i + 1]

            if isinstance(next_step, str):
                input_sequences.append((tokenizer.texts_to_sequences(sequence), tokenizer.texts_to_sequences([next_step])))
            else:
                # Ignore non-string next steps
                pass


X = np.array([item[0][0] for item in input_sequences])
y = np.array([item[1][0][0] for item in input_sequences])'''

# Pad sequences
X = pad_sequences(X, padding='post')
y = np.expand_dims(y, axis=-1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the sequence-to-sequence model with attention
input_layer = Input(shape=(X.shape[1],))
embedding_layer = Embedding(input_dim=total_words, output_dim=50, input_length=X.shape[1])(input_layer)
lstm_layer = LSTM(100, return_sequences=True)(embedding_layer)
attention_layer = Attention()([lstm_layer, lstm_layer])
output_layer = Dense(total_words, activation='softmax')(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Now, you can use the trained model for predictions
# For example, if you have a new set of steps, you can predict the next steps
new_steps = ["Delivery", "Exploitation"]
new_steps_seq = [tokenizer.texts_to_sequences(new_steps)]
new_steps_padded = pad_sequences(new_steps_seq, maxlen=X.shape[1], padding='post')

# Predict the next steps
predicted_steps = model.predict(new_steps_padded)
predicted_next_step = tokenizer.index_word[np.argmax(predicted_steps)]

# Print the predicted next steps
print(f"Predicted next step: {predicted_next_step}")


# decision tree


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv('/kaggle/input/crime-data/crime data.csv')
data.drop(columns=['Process 9', 'Process 10', 'Process 11'])

crime_types = data.iloc[:, 1:2]
crime_types = crime_types.values.tolist()

crime_process_steps = data.iloc[:, 2:].values.tolist()
print("Length of flattened_crime_types:", len(flattened_crime_types))
print("Length of crime_process_steps:", len(crime_process_steps))


lowercase_steps = [[step.lower() if isinstance(step, str) else str(step) for step in sublist] for sublist in crime_process_steps]

flattened_steps = [step for sublist in lowercase_steps for step in sublist]

vectorizer = TfidfVectorizer()
crime_process_vectors = vectorizer.fit_transform(flattened_steps)

flattened_crime_types = [crime_types[i][0] for i in range(len(crime_process_steps))]

dtc = DecisionTreeClassifier()
dtc.fit(crime_process_vectors, flattened_crime_types)


def predict_next_step(crime_type, crime_process_steps):
    # Assuming you have previously defined vectorizer, dtc, and data

    crime_process_steps_vector = vectorizer.transform(crime_process_steps)

    next_crime_type = dtc.predict(crime_process_steps_vector)[0]

    next_crime_processes = data[data['crime_type'] == next_crime_type].iloc[:, 2:].values.tolist()

    next_step = None
    for crime_process in next_crime_processes:
        for step in crime_process:
            if not pd.isna(step) and step not in crime_process_steps:
                next_step = step
                break

    return next_step


# Example usage
crime_type = 'Phishing'
crime_process_steps = ['Target Selection', 'Message Crafting']
next_step = predict_next_step(crime_type, crime_process_steps)
print(next_step)


from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file = 'otput.html')